# Подход к решению
Возможные подходы:
1. Разбить слова на слоги по правилам русского языка.  Слово будет "текстом", слоги будут играть роль "слов". Дальше используем стандартные средства работы с текстом (векторизация и эмбеддинг слогов и т.п.).
2. Разбить слова на буквы, делать векторизацию букв. Алгоритм сам будет выявлять закономерности

В этой версии использую второй подход.

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import keras_tuner
import chime

2023-04-02 16:43:32.317057: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-02 16:43:32.615848: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-02 16:43:34.154433: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
%load_ext chime

In [3]:
df = pd.read_csv('train.csv', index_col='id'); df

,word,stress,num_syllables,lemma
id,,,,
0,румяной,2,3,румяный
1,цифрами,1,3,цифра
2,слугами,1,3,слуга
3,выбирает,3,4,выбирать
4,управдом,3,3,управдом
...,...,...,...,...
63433,экзамена,2,4,экзамен
63434,культурой,2,3,культура
63435,объемной,2,3,объемный


Для удобства сдвигаю нумерацию слогов, чтобы она начиналась с 0. Не забыть сдвинуть обратно окончательный прогноз!

In [4]:
df.stress = df.stress - 1

# Набор символов
Нахожу все используемые символы

In [5]:
def add_char(word):
    global alphabeth
    alphabeth = alphabeth.union(set(list(word)))

In [6]:
alphabeth = set()

In [7]:
df.word.apply(add_char)

id
0        None
1        None
2        None
3        None
4        None
         ... 
63433    None
63434    None
63435    None
63436    None
63437    None
Name: word, Length: 63438, dtype: object

In [8]:
df.lemma.apply(add_char)

id
0        None
1        None
2        None
3        None
4        None
         ... 
63433    None
63434    None
63435    None
63436    None
63437    None
Name: lemma, Length: 63438, dtype: object

Нумерую буквы

In [9]:
numbers = list(range(1, len(alphabeth) + 1))

In [10]:
alphabeth = {k: v for k, v in zip(list(alphabeth), numbers)}; alphabeth

{'ё': 1,
 'д': 2,
 'ж': 3,
 'в': 4,
 'т': 5,
 'э': 6,
 'б': 7,
 'н': 8,
 'л': 9,
 'з': 10,
 'ъ': 11,
 'ь': 12,
 'ш': 13,
 'м': 14,
 'й': 15,
 'р': 16,
 'к': 17,
 'п': 18,
 'о': 19,
 'ч': 20,
 'ы': 21,
 'ф': 22,
 'у': 23,
 'ю': 24,
 'е': 25,
 'х': 26,
 'а': 27,
 'я': 28,
 'г': 29,
 'с': 30,
 'щ': 31,
 'и': 32,
 'ц': 33}

# Подготовка данных 

Число символов и слова отдельно, т.к. подаю на разные входы нейронной сети

In [11]:
y = df.stress
num_syllables = df.num_syllables
X = df.drop(columns=['stress', 'num_syllables'])

Разделяем на буквы

In [12]:
word = X.word.apply(lambda x: ' '.join(list(x)))
lemma = X.lemma.apply(lambda x: ' '.join(list(x)))

In [13]:
word = word.str.split(expand=True); word

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
id,,,,,,,,,,,,,,,,,,,,
0,р,у,м,я,н,о,й,None,None,None,None,None,None,None,None,None,None,None,None,None
1,ц,и,ф,р,а,м,и,None,None,None,None,None,None,None,None,None,None,None,None,None
2,с,л,у,г,а,м,и,None,None,None,None,None,None,None,None,None,None,None,None,None
3,в,ы,б,и,р,а,е,т,None,None,None,None,None,None,None,None,None,None,None,None
4,у,п,р,а,в,д,о,м,None,None,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63433,э,к,з,а,м,е,н,а,None,None,None,None,None,None,None,None,None,None,None,None
63434,к,у,л,ь,т,у,р,о,й,None,None,None,None,None,None,None,None,None,None,None
63435,о,б,ъ,е,м,н,о,й,None,None,None,None,None,None,None,None,None,None,None,None


In [14]:
lemma = lemma.str.split(expand=True); lemma

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
id,,,,,,,,,,,,,,,,,,,,
0,р,у,м,я,н,ы,й,None,None,None,None,None,None,None,None,None,None,None,None,None
1,ц,и,ф,р,а,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,с,л,у,г,а,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3,в,ы,б,и,р,а,т,ь,None,None,None,None,None,None,None,None,None,None,None,None
4,у,п,р,а,в,д,о,м,None,None,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63433,э,к,з,а,м,е,н,None,None,None,None,None,None,None,None,None,None,None,None,None
63434,к,у,л,ь,т,у,р,а,None,None,None,None,None,None,None,None,None,None,None,None
63435,о,б,ъ,е,м,н,ы,й,None,None,None,None,None,None,None,None,None,None,None,None


Отсутсвующая буква в конце слова - 0. (Альтернативный вариант - ragged tensors.)

In [15]:
word.fillna(0, inplace=True)
lemma.fillna(0, inplace=True)

In [16]:
alphabeth[0] = 0

Заменяю буквы на номера

In [17]:
word = word.applymap(lambda x: alphabeth[x])
lemma = lemma.applymap(lambda x: alphabeth[x])

In [18]:
lemma

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
id,,,,,,,,,,,,,,,,,,,,
0,16,23,14,28,8,21,15,0,0,0,0,0,0,0,0,0,0,0,0,0
1,33,32,22,16,27,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,30,9,23,29,27,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,4,21,7,32,16,27,5,12,0,0,0,0,0,0,0,0,0,0,0,0
4,23,18,16,27,4,2,19,14,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63433,6,17,10,27,14,25,8,0,0,0,0,0,0,0,0,0,0,0,0,0
63434,17,23,9,12,5,23,16,27,0,0,0,0,0,0,0,0,0,0,0,0
63435,19,7,11,25,14,8,21,15,0,0,0,0,0,0,0,0,0,0,0,0


Объединяю

In [19]:
X = np.concatenate((word, lemma), axis=1)

# Модель

In [20]:
callbacks = [
    keras.callbacks.EarlyStopping(
        monitor="val_accuracy",
        min_delta=0,
        patience=10,
        verbose=1,
        mode="max",
        baseline=None,
        restore_best_weights=True,
        start_from_epoch=10
    )
    ]

In [21]:
def build_model(hp):
    embedding_dim = hp.Int('emb_dim', min_value=4, max_value=32, step=4)
    embedding_length = hp.Int('emb_len', min_value=4, max_value=32, step=4)
    lstm_1 = hp.Int('lstm_1', min_value=32, max_value=64, step=8)
    lstm_2 = hp.Int('lstm_2', min_value=32, max_value=64, step=8)
    hp_units = hp.Int('units', min_value=32, max_value=64, step=8)
    
    word_input = keras.Input(
        shape=(40),
        name='words_and_lemmas'
    )
    syll_input = keras.Input(
        shape=(1),
        name='num_syllables'
    )

    w = layers.Embedding(
        input_dim=34,
        output_dim=embedding_dim,
        mask_zero=True,
        input_length=embedding_length)(word_input)
    w = layers.Bidirectional(layers.LSTM(units=lstm_1, dropout=0.1, return_sequences=True))(w)
    w = layers.Bidirectional(layers.LSTM(units=lstm_2, dropout=0.1))(w)

    s = layers.Dense(1, activation='relu')(syll_input)

    x = layers.concatenate([w, s])
    x = layers.Dense(units=hp_units, activation='relu')(x)
    x = layers.Dropout(0.1)(x)

    pred = layers.Dense(6, activation='softmax')(x)

    model = keras.Model(
        inputs = [word_input, syll_input],
        outputs = [pred])
        
    model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              optimizer=keras.optimizers.Adam(),
              metrics='accuracy')
    return model    

In [25]:
tuner = keras_tuner.BayesianOptimization(
    hypermodel=build_model,
    objective='val_accuracy',
    max_trials=25,
    num_initial_points=5,
    seed=8)

INFO:tensorflow:Reloading Tuner from ./untitled_project/tuner0.json


In [26]:
%%time
%%chime
tuner.search(
    {'words_and_lemmas': X,
    'num_syllables': num_syllables},
    y,
    epochs=50,
    validation_split=0.2,
    callbacks=callbacks
)

Trial 25 Complete [00h 58m 36s]
val_accuracy: 0.8511191606521606

Best val_accuracy So Far: 0.9024274945259094
Total elapsed time: 14h 06m 37s
INFO:tensorflow:Oracle triggered exit
CPU times: user 1d 21h 56min 31s, sys: 46min 2s, total: 1d 22h 42min 34s
Wall time: 14h 6min 38s


In [27]:
tuner.results_summary()

Results summary
Results in ./untitled_project
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 16 summary
Hyperparameters:
emb_dim: 32
emb_len: 32
lstm_1: 64
lstm_2: 32
units: 64
Score: 0.9024274945259094

Trial 13 summary
Hyperparameters:
emb_dim: 32
emb_len: 4
lstm_1: 64
lstm_2: 64
units: 32
Score: 0.9023486971855164

Trial 08 summary
Hyperparameters:
emb_dim: 28
emb_len: 20
lstm_1: 64
lstm_2: 64
units: 48
Score: 0.9005359411239624

Trial 23 summary
Hyperparameters:
emb_dim: 32
emb_len: 16
lstm_1: 64
lstm_2: 64
units: 32
Score: 0.9005359411239624

Trial 09 summary
Hyperparameters:
emb_dim: 32
emb_len: 32
lstm_1: 64
lstm_2: 64
units: 32
Score: 0.8999842405319214

Trial 11 summary
Hyperparameters:
emb_dim: 32
emb_len: 4
lstm_1: 64
lstm_2: 64
units: 64
Score: 0.8990384340286255

Trial 10 summary
Hyperparameters:
emb_dim: 32
emb_len: 32
lstm_1: 64
lstm_2: 64
units: 64
Score: 0.8984867334365845

Trial 14 summary
Hyperparameters:
emb_dim: 32
emb_len: 4
lstm_1: 

Get the optimal hyperparameters

In [33]:
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

Build the model with the optimal hyperparameters and train it on the data for 100 epochs

In [35]:
model = tuner.hypermodel.build(best_hps)

In [36]:
history = model.fit(
    {'words_and_lemmas': X,
    'num_syllables': num_syllables},
    y,
    epochs=100, validation_split=0.2)

Epoch 1/100


2023-04-03 10:37:47.752118: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/ReverseV2_grad/ReverseV2/ReverseV2/axis' with dtype int32 and shape [1]
	 [[{{node gradients/ReverseV2_grad/ReverseV2/ReverseV2/axis}}]]
2023-04-03 10:37:48.940132: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/ReverseV2_grad/ReverseV2/ReverseV2/axis' with dtype int32 and shape [1]
	 [[{{node gradients/ReverseV2_grad/ReverseV2/ReverseV2/axis}}]]
2023-04-03 10:37:56.484148: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ign

1586/1586 [==============================] - 103s 52ms/step - loss: 0.8976 - accuracy: 0.5984 - val_loss: 0.7788 - val_accuracy: 0.6616
Epoch 2/100
1586/1586 [==============================] - 75s 48ms/step - loss: 0.7556 - accuracy: 0.6758 - val_loss: 0.6877 - val_accuracy: 0.7111
Epoch 3/100
1586/1586 [==============================] - 75s 47ms/step - loss: 0.6718 - accuracy: 0.7227 - val_loss: 0.6325 - val_accuracy: 0.7398
Epoch 4/100
1586/1586 [==============================] - 75s 47ms/step - loss: 0.6224 - accuracy: 0.7443 - val_loss: 0.5758 - val_accuracy: 0.7625
Epoch 5/100
1586/1586 [==============================] - 77s 49ms/step - loss: 0.5788 - accuracy: 0.7659 - val_loss: 0.5424 - val_accuracy: 0.7782
Epoch 6/100
1586/1586 [==============================] - 78s 49ms/step - loss: 0.5427 - accuracy: 0.7820 - val_loss: 0.5250 - val_accuracy: 0.7874
Epoch 7/100
1586/1586 [==============================] - 78s 49ms/step - loss: 0.5109 - accuracy: 0.7941 - val_loss: 0.4948 - val

1586/1586 [==============================] - 78s 49ms/step - loss: 0.1134 - accuracy: 0.9559 - val_loss: 0.3994 - val_accuracy: 0.8968
Epoch 57/100
1586/1586 [==============================] - 78s 49ms/step - loss: 0.1067 - accuracy: 0.9588 - val_loss: 0.3864 - val_accuracy: 0.8991
Epoch 58/100
1586/1586 [==============================] - 78s 49ms/step - loss: 0.1050 - accuracy: 0.9595 - val_loss: 0.3954 - val_accuracy: 0.9034
Epoch 59/100
1586/1586 [==============================] - 78s 49ms/step - loss: 0.1042 - accuracy: 0.9599 - val_loss: 0.4069 - val_accuracy: 0.8988
Epoch 60/100
1586/1586 [==============================] - 78s 49ms/step - loss: 0.1010 - accuracy: 0.9613 - val_loss: 0.4019 - val_accuracy: 0.9001
Epoch 61/100
1586/1586 [==============================] - 78s 49ms/step - loss: 0.1015 - accuracy: 0.9619 - val_loss: 0.3890 - val_accuracy: 0.9017
Epoch 62/100
1586/1586 [==============================] - 78s 49ms/step - loss: 0.1001 - accuracy: 0.9613 - val_loss: 0.3945 

In [37]:
val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

Best epoch: 84


In [38]:
hypermodel = tuner.hypermodel.build(best_hps)

In [39]:
hypermodel.fit(
    {'words_and_lemmas': X,
    'num_syllables': num_syllables},
    y,
    epochs=best_epoch)

Epoch 1/84


2023-04-03 13:42:28.602276: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/ReverseV2_grad/ReverseV2/ReverseV2/axis' with dtype int32 and shape [1]
	 [[{{node gradients/ReverseV2_grad/ReverseV2/ReverseV2/axis}}]]
2023-04-03 13:42:29.857989: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/ReverseV2_grad/ReverseV2/ReverseV2/axis' with dtype int32 and shape [1]
	 [[{{node gradients/ReverseV2_grad/ReverseV2/ReverseV2/axis}}]]
2023-04-03 13:42:38.214466: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ign

1983/1983 [==============================] - 112s 46ms/step - loss: 0.8786 - accuracy: 0.6090
Epoch 2/84
1983/1983 [==============================] - 90s 45ms/step - loss: 0.7371 - accuracy: 0.6868
Epoch 3/84
1983/1983 [==============================] - 90s 45ms/step - loss: 0.6724 - accuracy: 0.7202
Epoch 4/84
1983/1983 [==============================] - 90s 45ms/step - loss: 0.6197 - accuracy: 0.7413
Epoch 5/84
1983/1983 [==============================] - 89s 45ms/step - loss: 0.5742 - accuracy: 0.7621
Epoch 6/84
1983/1983 [==============================] - 89s 45ms/step - loss: 0.5339 - accuracy: 0.7804
Epoch 7/84
1983/1983 [==============================] - 91s 46ms/step - loss: 0.5018 - accuracy: 0.7944
Epoch 8/84
1983/1983 [==============================] - 91s 46ms/step - loss: 0.4724 - accuracy: 0.8067
Epoch 9/84
1983/1983 [==============================] - 91s 46ms/step - loss: 0.4465 - accuracy: 0.8196
Epoch 10/84
1983/1983 [==============================] - 92s 46ms/step - l

1983/1983 [==============================] - 92s 46ms/step - loss: 0.0893 - accuracy: 0.9662
Epoch 80/84
1983/1983 [==============================] - 93s 47ms/step - loss: 0.0887 - accuracy: 0.9674
Epoch 81/84
1983/1983 [==============================] - 92s 46ms/step - loss: 0.0866 - accuracy: 0.9681
Epoch 82/84
1983/1983 [==============================] - 92s 46ms/step - loss: 0.0876 - accuracy: 0.9678
Epoch 83/84
1983/1983 [==============================] - 92s 46ms/step - loss: 0.0873 - accuracy: 0.9677
Epoch 84/84
1983/1983 [==============================] - 92s 46ms/step - loss: 0.0859 - accuracy: 0.9682


# Прогноз на тестовой выборке 

## Подготовка данных 

In [40]:
test = pd.read_csv('test.csv', index_col='id'); test

,word,num_syllables,lemma
id,,,
0,эпилепсия,5,эпилепсия
1,относящейся,5,относиться
2,размышлениями,6,размышление
3,модемы,3,модем
4,солнц,1,солнце
...,...,...,...
29955,донбасса,3,донбасс
29956,обложка,3,обложка
29957,правителя,4,правитель


Число символов и слова отдельно, т.к. подаю на разные входы нейронной сети

In [41]:
num_syllables_test = test.num_syllables
X_test = test.drop(columns=['num_syllables'])

Разделяем на буквы

In [42]:
word_test = X_test.word.apply(lambda x: ' '.join(list(x)))
lemma_test = X_test.lemma.apply(lambda x: ' '.join(list(x)))

In [43]:
word_test = word_test.str.split(expand=True); word_test

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
id,,,,,,,,,,,,,,,,,,,
0,э,п,и,л,е,п,с,и,я,None,None,None,None,None,None,None,None,None,None
1,о,т,н,о,с,я,щ,е,й,с,я,None,None,None,None,None,None,None,None
2,р,а,з,м,ы,ш,л,е,н,и,я,м,и,None,None,None,None,None,None
3,м,о,д,е,м,ы,None,None,None,None,None,None,None,None,None,None,None,None,None
4,с,о,л,н,ц,None,None,None,None,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29955,д,о,н,б,а,с,с,а,None,None,None,None,None,None,None,None,None,None,None
29956,о,б,л,о,ж,к,а,None,None,None,None,None,None,None,None,None,None,None,None
29957,п,р,а,в,и,т,е,л,я,None,None,None,None,None,None,None,None,None,None


In [44]:
lemma_test = lemma_test.str.split(expand=True); lemma_test

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
id,,,,,,,,,,,,,,,,,,,
0,э,п,и,л,е,п,с,и,я,None,None,None,None,None,None,None,None,None,None
1,о,т,н,о,с,и,т,ь,с,я,None,None,None,None,None,None,None,None,None
2,р,а,з,м,ы,ш,л,е,н,и,е,None,None,None,None,None,None,None,None
3,м,о,д,е,м,None,None,None,None,None,None,None,None,None,None,None,None,None,None
4,с,о,л,н,ц,е,None,None,None,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29955,д,о,н,б,а,с,с,None,None,None,None,None,None,None,None,None,None,None,None
29956,о,б,л,о,ж,к,а,None,None,None,None,None,None,None,None,None,None,None,None
29957,п,р,а,в,и,т,е,л,ь,None,None,None,None,None,None,None,None,None,None


Отсутсвующая буква в конце слова - 0. (Альтернативный вариант - ragged tensors.)

In [45]:
word_test.fillna(0, inplace=True)
lemma_test.fillna(0, inplace=True)

Заменяю буквы на номера

In [46]:
word_test = word_test.applymap(lambda x: alphabeth[x])
lemma_test = lemma_test.applymap(lambda x: alphabeth[x])

In [47]:
lemma_test

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
id,,,,,,,,,,,,,,,,,,,
0,6,18,32,9,25,18,30,32,28,0,0,0,0,0,0,0,0,0,0
1,19,5,8,19,30,32,5,12,30,28,0,0,0,0,0,0,0,0,0
2,16,27,10,14,21,13,9,25,8,32,25,0,0,0,0,0,0,0,0
3,14,19,2,25,14,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,30,19,9,8,33,25,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29955,2,19,8,7,27,30,30,0,0,0,0,0,0,0,0,0,0,0,0
29956,19,7,9,19,3,17,27,0,0,0,0,0,0,0,0,0,0,0,0
29957,18,16,27,4,32,5,25,9,12,0,0,0,0,0,0,0,0,0,0


Объединяю

In [71]:
extra_column = np.zeros((num_syllables_test.shape[0], 1), dtype=int)

In [72]:
X_test = np.concatenate((word_test, 
                         extra_column,
                         lemma_test,
                         extra_column
                        ), axis=1); X_test

array([[ 6, 18, 32, ...,  0,  0,  0],
       [19,  5,  8, ...,  0,  0,  0],
       [16, 27, 10, ...,  0,  0,  0],
       ...,
       [18, 16, 27, ...,  0,  0,  0],
       [13, 25, 16, ...,  0,  0,  0],
       [19, 18,  5, ...,  0,  0,  0]])

## Прогноз 

In [81]:
y_test_pred_arr = hypermodel.predict({'words_and_lemmas': X_test,
    'num_syllables': num_syllables_test})

937/937 [==============================] - 12s 13ms/step


In [83]:
y_test_pred = y_test_pred_arr.argmax(axis=1)

Проверка на разумность: номер предсказанного ударного слога не должен быть больше числа слогов в слове.

In [84]:
(y_test_pred + 1 > num_syllables_test).sum()

28

In [79]:
submission = pd.DataFrame(data=y_test_pred, index=test.index, columns=['stress'])
submission.stress = submission.stress + 1
submission

,stress
id,
0,3
1,3
2,3
3,2
4,1
...,...
29955,2
29956,2
29957,2


In [80]:
submission.to_csv('submission_06.csv')

### Исправление тех значений, где предсказанный номер слога больше числа слогов в слове 

In [91]:
results = np.concatenate((y_test_pred_arr, np.expand_dims(num_syllables_test, axis=1)), axis=1)

In [111]:
y_test_pred_2 = np.apply_along_axis(lambda x: np.argmax(x[0 : int(x[6])]), axis=1, arr=results)

In [113]:
submission_2 = pd.DataFrame(data=y_test_pred_2, index=test.index, columns=['stress'])
submission_2.stress = submission_2.stress + 1
submission_2

,stress
id,
0,3
1,3
2,3
3,2
4,1
...,...
29955,2
29956,2
29957,2


In [116]:
submission_2.to_csv('submission_06_2.csv')